In [45]:
import pandas as pd
import seaborn as sns
import openpyxl
import math
import re
import numpy as np

In [98]:
zipdata = pd.read_csv("ExternalDataByZipcode.csv")

# convert data into usuable types
zipdata['Median Household Income'] = zipdata['Median Household Income'].str.replace('\W', '', regex=True).astype(float)
zipdata['Population'] = zipdata['Population'].str.replace('\W', '', regex=True).astype(float)

# print(zipdata.var())
# print(zipdata.dtypes)
zipdata.head()

# for each zip code find standardized euclidean norm (2-norm) of varialbes
# vars = [income, age, population, nonwhite]

# initialize list of norms
norm = []

# select which zipcode to test
zipcode = 77009
zipindex = zipdata[zipdata['Zip Codesort column'] == zipcode].index.values
targeted = [zipindex[0]]

for precinct in targeted:
    # retrieves values of variables for zipcodes of targeted precincts
    tar_income = zipdata.iloc[precinct]['Median Household Income']
    tar_age = zipdata.iloc[precinct]['Median Age']
    tar_pop = zipdata.iloc[precinct]['Population']


    for i in range(zipdata.shape[0]):
        # takes out testing precinct with itself
        if precinct != i:
            # retrieves values of variables for other zipcodes
            income = zipdata.iloc[i]['Median Household Income']
            age = zipdata.iloc[i]['Median Age']
            pop = zipdata.iloc[i]['Population']
            # standardizes the difference between the targeted and other zipcodes
            # i.e. income and age are varied across vastly different scales
            incomediff = (tar_income - income) / (1.067543e+09)
            agediff = (tar_age - age) / (1.693905e+01)
            popdiff = (tar_pop - pop) / (1.558452e+08)
            # sums the standardized differences
            sum = ((incomediff)**2) + ((agediff)**2) + ((popdiff)**2)
        # takes the square root of the sum according to formula
        norm.append(math.sqrt(sum))

# print(norm)
print(min(norm))
indexnorm = np.argmin(norm)
print('targeted zipcode:', zipdata.loc[[targeted[0]]])
print('closest other zipcode:', zipdata.loc[[indexnorm]])

0.00019303190379615993
targeted zipcode:    Zip Codesort column  Median Household Income  Median Age  Population  \
7                77009                  64011.0        37.0     25649.0   

  White Percentage Black/AfricanAmerican Percentage  \
7           64.61%                            7.43%   

  Pacific Islander Percentage Asian Percentage Other Race Population  \
7                       0.07%            1.23%                21.86%   

  2+ Races Percentage     sum  
7               3.91%  99.11%  
closest other zipcode:      Zip Codesort column  Median Household Income  Median Age  Population  \
105                77379                 107167.0        37.0     55065.0   

    White Percentage Black/AfricanAmerican Percentage  \
105           64.54%                           11.50%   

    Pacific Islander Percentage Asian Percentage Other Race Population  \
105                       0.08%           12.30%                 6.59%   

    2+ Races Percentage     sum  
105         